# Process static tactile data
* Features are SA and FA firing over a single frame, saved as two arrays:
*        feature_SA and features_RA of shape (n_pins, n_pins)
* n_pins is number of pins along side of square array (here 19)
* Split into train, validation and test sets
* For each speed, make distinct training set with that speed left out but retained for testing
* Spatial/static uses single samples/frames. Labels are the texture (here 13)

To run, first edit dir_data to path where data is stored; preprocess/process should be run first. 

Because of the large amount of data, a temporary folder is used for the training data, so it can be deleted after training the static model.

In [1]:
import os, pickle
import numpy as np

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def open_obj(name):
    with open(name + '.pkl', 'rb') as f:
        obj = pickle.load(f)
    return obj

In [2]:
dir_data = os.environ["DATAPATH"] + r"/open/afferents-tactile-textures-jrsi2022"
dir_temp = os.environ["TEMPPATH"] 

dir_train = dir_temp + r"/static"
dir_test = dir_data + r"/static"

speeds = ['10','20','30','40','50','60','70','80','90','100']
textures = ['0','0.5','1','1.5','2','2.5','3','3.5','4','4.5','5','5.5','6']
n_textures = len(textures)
n_speeds = len(speeds)

firing_SA = open_obj(dir_data + r"/firing_SA")
firing_RA = open_obj(dir_data + r"/firing_RA")
pins = open_obj(dir_data + rf"/pins")

In [3]:
def rotate_pins(pins, theta):
    R = np.array(((np.cos(theta),-np.sin(theta)), (np.sin(theta), np.cos(theta))))
    pins_rot = [np.matmul(R, pin) for pin in pins]
    return np.array(pins_rot)

def locate_pins(pins, n_pins):
    locs = np.zeros((n_pins, n_pins))
    index_xSort = np.argsort(pins[:,0], axis=0)
    
    index_ySort = np.zeros((n_pins, n_pins))
    n = 0
    for k in range(n_pins):
        indices = np.zeros((n_pins,2))
        for l in range(n_pins):
            indices[l,:] = pins[index_xSort[n],:]
            n+=1
        index_ySort[k,:] = np.argsort(indices[:,1], axis=0)

    for k in range(n_pins):
        for l in range(n_pins):
            locs[k,l] = index_xSort[int(index_ySort[k,l])+n_pins*k]
    return locs

def features(firing, locs, i_start, n_samples):
    n1_pins, n2_pins = np.shape(locs)
    features = np.zeros((n_samples, n1_pins, n2_pins))
    for k in range(n1_pins):
        for l in range(n2_pins):
            n = 0
            for m in range(i_start, i_start+n_samples):
                features[n,k,l] = firing[m, int(locs[k,l])]  
                n+=1   
    return features

In [4]:
theta = np.radians(-6.2)
n_pins = 19
i_start = 500
n_samples = 5000 

pins_rot = [[rotate_pins(pins[i][j], theta) for  j in range(n_speeds)] for i in range(n_textures)]
locs_rot = [[locate_pins(pins_rot[i][j], n_pins) for  j in range(n_speeds)] for i in range(n_textures)]
features_SA = [[features(firing_SA[i][j], locs_rot[i][j], i_start, n_samples) for j in range(n_speeds)] for i in range(n_textures)]
features_RA = [[features(firing_RA[i][j], locs_rot[i][j], i_start, n_samples) for j in range(n_speeds)] for i in range(n_textures)]

X_SA = np.swapaxes(np.stack(features_SA), 0, 1) # speeds, textures,....
X_RA = np.swapaxes(np.stack(features_RA), 0, 1) # speeds, textures,....

In [5]:
n_train = n_samples*n_textures*(n_speeds-1) # leave one speed out
n_val = int(n_samples*n_textures/2)
n_test = int(n_samples*n_textures/2) # equal val test split

os.makedirs(dir_train, exist_ok=True)
os.makedirs(dir_test, exist_ok=True)

for i in range(n_speeds):
    dir_train_i = dir_train + rf"/{i}"
    dir_test_i = dir_test + rf"/{i}"
    os.makedirs(dir_train_i, exist_ok=True)
    os.makedirs(dir_test_i, exist_ok=True)

    X_train_SA, X_train_RA = (np.zeros((n_train, n_pins, n_pins, 1)) for _ in range(2))
    X_val_SA, X_val_RA = (np.zeros((n_val, n_pins, n_pins, 1)) for _ in range(2))
    X_test_SA, X_test_RA = (np.zeros((n_test, n_pins, n_pins, 1)) for _ in range(2))
    y_train_SA_texture, y_train_RA_texture = (np.zeros((n_train, n_textures)) for _ in range(2))
    y_train_SA_speed, y_train_RA_speed = (np.zeros((n_train, 1)) for _ in range(2))
    y_val_SA_texture, y_val_RA_texture = (np.zeros((n_val, n_textures)) for _ in range(2))
    y_val_SA_speed, y_val_RA_speed = (np.zeros((n_val, 1)) for _ in range(2))
    y_test_SA_texture, y_test_RA_texture = (np.zeros((n_test, n_textures)) for _ in range(2))
    y_test_SA_speed, y_test_RA_speed = (np.zeros((n_test, 1)) for _ in range(2))
    
    n = 0
    for j in range(n_speeds):
        if not j==i:
            X_train_SA[n*n_textures*n_samples:(n+1)*n_textures*n_samples,...] = np.expand_dims(np.concatenate(X_SA[j,...]), axis=3)
            X_train_RA[n*n_textures*n_samples:(n+1)*n_textures*n_samples,...] = np.expand_dims(np.concatenate(X_RA[j,...]), axis=3)
            for k in range(n_textures):
                y_train_SA_texture[(n*n_textures+k)*n_samples : (n*n_textures+k+1)*n_samples, k] = 1
                y_train_RA_texture[(n*n_textures+k)*n_samples : (n*n_textures+k+1)*n_samples, k] = 1
                y_train_SA_speed[(n*n_textures+k)*n_samples : (n*n_textures+k+1)*n_samples] = j/n_speeds
                y_train_RA_speed[(n*n_textures+k)*n_samples : (n*n_textures+k+1)*n_samples] = j/n_speeds
            n+=1
        else:
            val_ind = range(0,n_samples,2)#np.random.choice(N_samples, int(N_samples/2), replace=False)
            n_1 = 0
            n_2 = 0
            for k in range(n_samples):
                if k in val_ind:
                    X_val_SA[n_1*n_textures:(n_1+1)*n_textures,...] = np.expand_dims(X_SA[j,:,k,...], axis=3)
                    X_val_RA[n_1*n_textures:(n_1+1)*n_textures,...] = np.expand_dims(X_RA[j,:,k,...], axis=3)
                    for l in range(n_textures):
                        y_val_SA_texture[n_1*n_textures+l,l] = 1
                        y_val_RA_texture[n_1*n_textures+l,l] = 1
                        y_val_SA_speed[n_1*n_textures+l] = i/n_speeds
                        y_val_RA_speed[n_1*n_textures+l] = i/n_speeds
                    n_1+=1
                else:
                    X_test_SA[n_2*n_textures:(n_2+1)*n_textures,...] = np.expand_dims(X_SA[j,:,k,...], axis=3)
                    X_test_RA[n_2*n_textures:(n_2+1)*n_textures,...] = np.expand_dims(X_RA[j,:,k,...], axis=3)
                    for l in range(n_textures):
                        y_test_SA_texture[n_2*n_textures+l,l] = 1
                        y_test_RA_texture[n_2*n_textures+l,l] = 1
                        y_test_SA_speed[n_2*n_textures+l] = i/n_speeds
                        y_test_RA_speed[n_2*n_textures+l] = i/n_speeds
                    n_2+=1
        
    save_obj(X_train_SA, dir_train_i + r"/X_train_SA")
    save_obj(X_val_SA, dir_train_i + r"/X_val_SA")
    save_obj(X_test_SA, dir_test_i + r"/X_test_SA")

    save_obj([y_train_SA_speed, y_train_SA_texture], dir_train_i + r"/y_train_SA")
    save_obj([y_val_SA_speed, y_val_SA_texture], dir_train_i + r"/y_val_SA")
    save_obj([y_test_SA_speed, y_test_SA_texture], dir_test_i + r"/y_test_SA")

    save_obj(X_train_RA, dir_train_i + r"/X_train_RA")
    save_obj(X_val_RA, dir_train_i + r"/X_val_RA")
    save_obj(X_test_RA, dir_test_i + r"/X_test_RA")

    save_obj([y_train_RA_speed, y_train_RA_texture], dir_train_i + r"/y_train_RA")
    save_obj([y_val_RA_speed, y_val_RA_texture], dir_train_i + r"/y_val_RA")
    save_obj([y_test_RA_speed, y_test_RA_texture], dir_test_i + r"/y_test_RA")